In [ ]:
import pickle5 as pickle
with open('../../data/COVID-QA/top_N_ents_spacy-COVID_QA.pkl', 'rb') as f:
    top_N_ents = pickle.load(f)

In [ ]:
import wikipedia
from tqdm.auto import tqdm

search_res = {}

for i, ent in tqdm(enumerate(top_N_ents)):
    #Skipping those entities which don't return anything
    if wikipedia.search(ent) != []:
        search_res[ent] = wikipedia.search(str(ent), results=1)[0]

In [ ]:
import wikipedia
from transformers import AutoTokenizer, AutoModel
import torch

context_dict = {}

filtering = False
if filtering == True:
    filtering_threshold = 0.5
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #We're consider scibert because pubmedbert assigns very high similarity for both related/unrelated terms.
    checkpoint = 'allenai/scibert_scivocab_uncased'
    
    model = AutoModel.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    cos = torch.nn.CosineSimilarity(dim=0)
    model.to(device)

for ent,res in tqdm(search_res.items()):
    if filtering == False:
        try:
            context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
        except:
            continue
    else:
        encoded_input = tokenizer([ent, res], return_tensors='pt', padding=True)
        with torch.no_grad():
            output = model(**encoded_input)
        
        similarity = cos(output.pooler_output[0], output.pooler_output[1])
        '''
        we're taking less than here since the similarity scores for related terms seem to be lower than 
        unrelated ones.
        '''
        if similarity.item() < filtering_thresold:
            try:
                context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
            except:
                continue

In [ ]:
import pandas as pd
pd.DataFrame(context_dict.items(), columns = ['ent', 'text']).to_parquet('wiki_corpus.parquet', index=False)

In [ ]:
from pymed import PubMed

# Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Create a GraphQL query in plain text
query = ['covid-19', 'hiv-1']

# Execute the query against the API
results = pubmed.query(query, max_results=500)

# Loop over the retrieved articles
for article in results:

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title
    abstract = article.abstract

    # Show information about the article
    print(
        f'{abstract}\n'
    )

In [ ]:
from datasets import load_dataset
s = load_dataset('json', data_files='../../data/RadQA'\
                 '/radqa-a-question-answering-dataset-to-improve-comprehension-of-radiology-reports-1.0.0'\
                 '/train.json', field='data')

In [ ]:
all_contexts = []
all_q = []
for row in s['train']['paragraphs']:
    for d in row:
        all_contexts.append(d["context"])
        for q in d['qas']:
            all_q.append(q["question"])

In [ ]:
import re
c=0
for r in all_contexts:
    #if re.search('findings',r, re.IGNORECASE) and re.search('impression:', r, re.IGNORECASE):
    #if re.search('final report', r, re.IGNORECASE) and re.search('impression:', r, re.IGNORECASE):
    if re.search('findings and impression', r, re.IGNORECASE):
        print(r)
        print('-'*100)
        

In [ ]:
a = 0
for row in s['train']['paragraphs']:
    for c in row:
        if c['context'] == 'IMPRESSION':
            print(c)
print(a)

In [42]:
import pickle5 as pickle
with open('../../data/RadQA/ents_spacy_radqa.pkl', 'rb') as f:
    s = pickle.load(f)

In [44]:
for i in s:
    print(i.strip())

repeat
 radiograph
problem
Cortical
left calf
 veins
pulmonary congestion
VIEWS.No
right lower lung
leukocytosis
dense stool
photon
 counts
Occipital bone fracture
wheeze
cystic mass
worsening tachypnea
bronchocentric
 infection
Portable technique
CVVH
axillary surgery
portion
Follow up fracture
heart post
supervening pneumonia
ovarian tumor
Clinical correlation
fact laceration
microscopic hematuria
duodenal closure
distal superior vena
 cava
chronic kidney
 disease
coronary arterial calcification
peak systolic velocity
filled masses
articular surface
reconstitution
bony
mass effect
Decrease
right lower
 extremity veins
right
 neck
small
 amount
right abdomen
Atrial fibrillation
median
 sternotomy
distal
Sequences
enhancement/edema
dual-lead
 cardiac pacemaker
septic source
airways
question pulmonary
 edema
involutional change
solid hypoechoic mass
tissue samples
status post coronary artery
 bypass graft surgery
Locally advanced ovarian carcinoma
cardiac variation
hydronephrosis
Humera

left proximal thigh
right lamina paprycea fracture
segmental areas
left internal
 carotid artery
left Internal jugular properly
traumatic aortic injury
penetration
portable chest
 radiograph from [**2122-2-5**]
chronic lung disease
Paraganglioma
[**2176-7-3**]
tube placement
mesenteric stranding
intestinal obstruction
right costophrenic angle
surgeries
left
 superficial femoral vein
tooth extraction
bridging
right-sided
calcaneal
colonic
obstructive pulmonary disease
non
 loculated
bleed
transient
document
multiplanar
report
[**2101-3-24**]
[**3-18**]:

 Heterogeneous opacification
localized
distal ureters
right internal jugular
 central
axillary lymph adenopathy
diagnosis
pre-operative
 localization
bloody sputum
comminuted
 fracture
thin barium
intermediate density
unchanged
subluxation
axillary lymph
 nodes
bilateral
retropharynx
entrapment
cervical vessels
anterior
 lower
Gallstones
right parafalcine subdural hematoma
right internal
thoracic
left sided chest tube
poles
imaging
knee

gas
mammogrpahy
Ejection fraction
Anomalous
left anterior clinoid region
right ventricular lead
Stool
left scarring
meningioma
left paramedian acute
 infarct
left
 transverse
[**2184-8-9**]
pleural effusion change
septal thickening
Foley trauma
costochondral portion
small polyp
mastoid air
 cells
right axilla
 (300b:35)
hypodense left thyroid lobe
 nodule
lung picture
chest x-ray
nasogastric tube position
biliary tract look
AM
  Mildly
congestive heart
 failure
right lung base
chronic
 microvascular infarction
male
annular
 tear
residual
 plate-like atelectasis
retrocardiac regions
venous gas
acquisitions
upper right paratracheal node
c1-C2
satisfactory
small vessel ischemic
excision
right internal jugular line tip
improvement
sparse
basal atelectasis
non-contrast head CT
dressed
decompression
pelvis
ex
 vacuo dilatation of frontal [**Doctor
minute
transaxial dimension
tubing
 catheters
Axial noncontrast CT scans
calcified menisci
Sinus

 Quantitative Measurements
 Measurement	Result	


Mr. [**Known lastname 37424**]
interpretation
left pulmonary artery
disc disease
PICC
Right upper lobe cicatricial
left vertebral artery
 directly
left base
Left subclavian
preeclampsia status post
 C-section
right transfemoral approach
cecum
overlying
 atelectasis
attenuation
short axis
right kidney
arrival
proptosis
positioning
ENYa
intra-cardiac shunt
[**2138-9-14**]
intracranial
lytic
Posterior fossa
right
 tenth
long term IV
internal wire
No pneumocephalus
duodenotomy
CTA/CTV
right sternocleidomastoid
 muscle
overlap
lumbar
 spine
AngioDynamics
premature spillage
Swan-Ganz catheter
gram
right tibial plateau
intraabdominal infection
sagittal axes
unchanged
O2
Written informed consent
right iliac crest
claudication
left brachial venous approach
Bilateral moderate
spleen measures
[**2115-1-17**]
procedure
IV contrast
 ______________________________________________________________________________
                                 FINAL REPORT
 Patient
right lateral
  ventricle
left pon

Multiple priors
T5/6
Monitoring
POST-PYLORIC
No
parenchymal abnormalities
years
FINDINGS
Uncomplicated ultrasound
outside
mild
 enhancement
post
 operative change
soft tissue abnormality
mid
 trachea
large bowel
frontal bone
[**2170-6-28**]:

 FINDINGS
cervical
 airway
dense mass
Soft tissue density right
Follow up
side
subareolar mass
atelectatic change
fluoroscopic image
interval volumes
S/P AVR
INDICATION
small-bowel injury
tip
 terminating
enteral tube
left kidneys
New placement right
right complex
axial plane
Questioned right
 upper pleural
poorly
abnormal
 enhancement
Pain
left-sided disc herniation
extraluminal air
pigtail catheter overlying
Right PICC line
extra-axial hemorrhage
atelectasis
area around pancreas
air pocket
small right pleural effusion persists
[**2138-9-20**]:

 Mild-to-moderate pulmonary edema
normal course
activity
left transverse
 sinus
MV repair
dialysis catheter
inferior
cervical soft tissue
interval
chest
 radiographs
time
MDCT-acquired contiguous images
c

three-vessel coronary artery calcifications
HOURS
sign
wire
 abscess
ultrasound
 criteria
right lower limb bones free
right-sided chest tube
pulmonary hemorrhage
encephalomalacia
radiographic
skin thickening
right cervical lymphadenopathy
malformation
Left upper extremity edema
Lytic
[**2191-8-4**]
abdominal process
goes
wound dehiscence
Portions
right-sided intraparenchymal
Ventral hernia repair
association
biliary leak
1.8-cm cyst
woman status post hernia repair
diminished
check Dobbhoff
paraaortic
volume
para-vertebral soft tissue component
proximal
 femur
knee
swallow pathology note
Multilevel
left paramedian hernia
costophrenic sulci
small bowels
tunnel
micropuncture set
Hepatic
Aeration
compressive atelectasis
central regions
status
 post chest tube discontinuation
aortic knob
emphysema
 ______________________________________________________________________________
                                 FINAL REPORT
 INDICATION
bedside thoracentesis
G tube patent
craniocaudal distance
